2.计算相似度：

1)相似度度量：

2)获得相似样本：得到每个历史样本与待测样本最相近的片段，选择最相近的30个历史样本

3)RUL估计：加权核密度估计 对最近的历史样本的相似度进行min-max归一化处理作为权重，加权得到样本的RUL

## 1 Import library

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import math
from sklearn.model_selection import train_test_split
from fastdtw import fastdtw
import tslearn.metrics #DTW
from scipy.spatial.distance import euclidean,correlation,cosine,cdist

## 读取数据

In [2]:
processed_data_file = '1_processed_data'
os.path.exists(processed_data_file)

True

In [3]:
tot_train_df = pd.read_csv(processed_data_file+'/train_PCA_data.csv',index_col=0)

In [4]:
tot_train_df.head()

,cycle,s2,s3,s4,s7,s11,s12,s15,RUL,HI
id,,,,,,,,,,
1,1,-0.934031,-0.518202,-0.815417,0.909791,-0.889393,1.011533,-0.829887,191,-2.246080
1,2,-0.928596,-0.523217,-0.821319,0.906845,-0.898700,1.004192,-0.833643,190,-2.249248
1,3,-0.918112,-0.532695,-0.832544,0.901018,-0.916229,0.989628,-0.840494,189,-2.254698
1,4,-0.903251,-0.545631,-0.848009,0.892464,-0.939992,0.968152,-0.849236,188,-2.260837
1,5,-0.884803,-0.560742,-0.866250,0.881470,-0.967400,0.940388,-0.858336,187,-2.265682


In [5]:
tot_cycles = pd.DataFrame([len(tot_train_df.loc[i]) for i in tot_train_df.index.unique()],columns =['tot_cycle'])
tot_cycles['rul_label'] = pd.DataFrame(pd.qcut(tot_cycles['tot_cycle'],4, labels=['1','2','3','4']))
tot_cycles.index = tot_train_df.index.unique()#range(1,101)

In [6]:
random_state = 1
tot_train_id = tot_train_df.index.unique()
# stratify 按照数据分布进行划分
train_id, test_id = train_test_split(tot_train_id, test_size = 0.2, random_state=random_state, stratify = tot_cycles['rul_label'])
train_df = tot_train_df.loc[train_id]
test_df = tot_train_df.loc[test_id]

In [7]:
test_id

Int64Index([11, 20, 17, 41, 42, 97, 80, 45, 48, 39, 36, 14, 31, 43, 29, 76, 16,
            55, 32, 24],
           dtype='int64', name='id')

In [8]:
selected_ftr_name = ['HI']

## 计算相似度

### 计算等长序列的DTW距离

In [9]:
#DTW
MAX_DIST=99
sim_num = 30
on_win_size = 30
on_win_stride = 10
his_win_size = on_win_size
his_win_stride = 1
scale = 1#.5
ftr_len = len(selected_ftr_name)

In [10]:
res_file = '2_res'
if os.path.exists(res_file)==False:
    os.makedirs(res_file)
#dtw_dist_matrix = np.load(res_file+'dtw_dist_matrix.npy')

### 计算欧式距离

In [16]:
dist_info_col = ['on_id','his_id','on_seg','his_seg','on_RUL','his_RUL','dist']
eu_dist_df = pd.DataFrame(columns=dist_info_col)
N_on_segs = {}
t_begin = time.time()
#dtw_dist_dict = {}
for on_id in test_id:
    t0 = time.time()
    on_unit = test_df.loc[on_id]
    on_len = len(on_unit)
    print('---',on_id,'---')
    n_segs = int((on_len - on_win_size)/on_win_stride+1)
    N_on_segs[on_id] = n_segs
    if on_len<on_win_size:
        continue
    #dtw_dist_dict[on_id] = {}
    
    for his_id in train_id:
        #dtw_dist_dict[on_id][his_id] = {}
        his_unit = train_df.loc[his_id]
        his_len = len(his_unit)
        cur_his_seg = 0
        for on_seg in range(0, on_len - on_win_size, on_win_stride):
            #dtw_dist_dict[on_id][his_id][on_seg] = {}
            dist_list = []
            for his_seg in range(cur_his_seg, his_len - his_win_size):
                on_array = on_unit[on_seg : on_seg + on_win_size][selected_ftr_name]
                his_array = his_unit[his_seg : his_seg + his_win_size][selected_ftr_name]
                dist_list += [euclidean(on_array, his_array)]
            min_his_seg = np.argsort(dist_list)[0]
            eu_dist_df = eu_dist_df.append(pd.DataFrame([[on_id, his_id, on_seg, min_his_seg+cur_his_seg, on_len - on_seg - on_win_size, his_len - (min_his_seg+cur_his_seg) - his_win_size,dist_list[min_his_seg]]],columns = dist_info_col))
            #dtw_dist_dict[on_id][his_id][on_seg] = (min_his_seg+cur_his_seg, dist_list[min_his_seg])
            cur_his_seg = max(min_his_seg+cur_his_seg - 20, 0)
            #print(his_id, on_seg, cur_his_seg)
            
    t1= time.time()
    print('unit_id:',on_id,'segs:',n_segs,'time:',t1-t0,'avg:',(t1-t0)/n_segs)
t_end = time.time()
print('tot_time:',t_end-t_begin)

--- 11 ---
unit_id: 11 segs: 22 time: 353.3406603336334 avg: 16.060939106074247
--- 20 ---
unit_id: 20 segs: 21 time: 206.60194396972656 avg: 9.838187808082218
--- 17 ---
unit_id: 17 segs: 25 time: 416.3046338558197 avg: 16.65218535423279
--- 41 ---
unit_id: 41 segs: 19 time: 304.62265610694885 avg: 16.03277137404994
--- 42 ---
unit_id: 42 segs: 17 time: 242.84213662147522 avg: 14.28483156596913
--- 97 ---
unit_id: 97 segs: 18 time: 291.1205995082855 avg: 16.173366639349197
--- 80 ---
unit_id: 80 segs: 16 time: 164.27591609954834 avg: 10.267244756221771
--- 45 ---
unit_id: 45 segs: 13 time: 237.88858819007874 avg: 18.299122168467594
--- 48 ---
unit_id: 48 segs: 21 time: 378.2005536556244 avg: 18.00955017407735
--- 39 ---
unit_id: 39 segs: 10 time: 106.9685845375061 avg: 10.696858453750611
--- 36 ---
unit_id: 36 segs: 13 time: 146.5989692211151 avg: 11.276843786239624
--- 14 ---
unit_id: 14 segs: 16 time: 163.64015340805054 avg: 10.227509588003159
--- 31 ---
unit_id: 31 segs: 21 time: 3

In [17]:
eu_dist_df.to_csv(res_file+'/eu_dist_'+str(random_state)+'.csv',index=False,header=True)

In [40]:
math.e**(-delta[delta>0]/13)

0     0.921103
0     0.731855
0     0.748054
0     0.742566
0     0.616551
0     0.743073
0     0.577489
0      0.29207
0     0.535422
0      0.74656
0    0.0174867
0    0.0403515
0     0.336136
0    0.0968499
0     0.298889
0     0.563359
0    0.0806627
0     0.192155
0     0.940165
0     0.467715
0     0.253509
0     0.180933
0     0.104014
0     0.685175
0      0.62109
0     0.293838
0      0.63994
0     0.958539
0     0.549544
0     0.717478
0     0.710207
0     0.752105
0     0.879973
0     0.158317
0     0.278852
0     0.147025
0       0.1028
0     0.787683
0     0.160626
0    0.0352921
0     0.290811
0    0.0598394
0     0.560784
0     0.721819
dtype: object

In [43]:
def RHS_score(y_true,y_pred):
    delta = y_pred-y_true
    s = sum(math.e**(-delta[delta<0]/13)-1) + sum(math.e**(delta[delta>0]/10)-1)
    return s

In [19]:
def get_score(pred_y,test_y):
    mse_test = np.sum((pred_y - test_y)**2)/len(test_y)
    rmse_test = mse_test**0.5
    mae_test = np.sum(np.absolute(pred_y - test_y))/len(test_y)
    r2_test = 1 - mse_test/np.var(test_y)
    rhs_test = RHS_score( test_y, pred_y)/len(pred_y)
    print('%.3f'%rmse_test)
    print('%.5f'%rhs_test)

In [22]:
param_grid = {
    'K': [i for i in range(50, 101, 10)],
    'alpha': [0, 0.5, 1, 1.5, 2]
}

In [42]:
math.e**(-delta[delta>0]/13).head()

0    0.921103
0    0.731855
0    0.748054
0    0.742566
0    0.616551
dtype: object

In [46]:
cut_percents = [0.5,0.7,0.9]
eu_score = []
for k in param_grid['K']:
    for alpha in param_grid['alpha']:
        res_info_col = ['on_id','on_seg','y_pred','y_true']
        res_df = pd.DataFrame(columns=res_info_col)
        for on_id in test_id:
            n_seg = len(test_df.loc[on_id])
            for cut_per in cut_percents:
                cur_seg = int(n_seg * cut_per/10)*10 - on_win_size
                tmp_df = eu_dist_df[(eu_dist_df['on_id']==on_id) &(eu_dist_df['on_seg']==cur_seg)].sort_values(by='dist')[:k]
                tmp_rul = sum(tmp_df['his_RUL']*np.exp(-alpha * tmp_df['dist'])/sum(np.exp(-alpha * tmp_df['dist'])))
                #print(cur_seg,tmp_rul, cur_seg+tmp_rul, n_seg - cur_seg - on_win_size)
                res_df= res_df.append(pd.DataFrame([[on_id, cur_seg, tmp_rul, n_seg -cur_seg - on_win_size]],columns = res_info_col))
        eu_score += [(k, alpha, RHS_score( res_df['y_true'], res_df['y_pred'])/len(res_df['y_pred']))]

In [47]:
eu_score

[(50, 0, 16.033057686517164),
 (50, 0.5, 7.641506521257928),
 (50, 1, 6.729597023605551),
 (50, 1.5, 7.3790699400709645),
 (50, 2, 9.77968687255469),
 (60, 0, 25.50458741574202),
 (60, 0.5, 8.953581879434088),
 (60, 1, 6.907890093431264),
 (60, 1.5, 7.319189382351793),
 (60, 2, 9.690991879984066),
 (70, 0, 43.09453404200685),
 (70, 0.5, 10.171817874028143),
 (70, 1, 7.065132763572955),
 (70, 1.5, 7.315140054423481),
 (70, 2, 9.6706490224621),
 (80, 0, 67.88780658639222),
 (80, 0.5, 11.03281022998642),
 (80, 1, 7.156030280051337),
 (80, 1.5, 7.317164614802855),
 (80, 2, 9.666531316404644),
 (90, 0, 67.88780658639222),
 (90, 0.5, 11.03281022998642),
 (90, 1, 7.156030280051337),
 (90, 1.5, 7.317164614802855),
 (90, 2, 9.666531316404644),
 (100, 0, 67.88780658639222),
 (100, 0.5, 11.03281022998642),
 (100, 1, 7.156030280051337),
 (100, 1.5, 7.317164614802855),
 (100, 2, 9.666531316404644)]

### Coarse: 计算DTW距离
对test片段从后往前依次遍历

In [25]:
dist_info_col = ['on_id','his_id','on_seg','his_seg','on_RUL','his_RUL','dist']
dtw_dist_df = pd.DataFrame(columns=dist_info_col)
N_on_segs = {}
t_begin = time.time()
#dtw_dist_dict = {}
for on_id in test_id:
    t0 = time.time()
    on_unit = test_df.loc[on_id]
    on_len = len(on_unit)
    print('---',on_id,'---')
    n_segs = int((on_len - on_win_size)/on_win_stride+1)
    N_on_segs[on_id] = n_segs
    if on_len<on_win_size:
        continue
    #dtw_dist_dict[on_id] = {}
    
    for his_id in train_id:
        #dtw_dist_dict[on_id][his_id] = {}
        his_unit = train_df.loc[his_id]
        his_len = len(his_unit)
        cur_his_seg = 0
        for on_seg in range(0, on_len - on_win_size, on_win_stride):
            #dtw_dist_dict[on_id][his_id][on_seg] = {}
            dist_list = []
            for his_seg in range(cur_his_seg, his_len - his_win_size):
                on_array = on_unit[on_seg : on_seg + on_win_size][selected_ftr_name]
                his_array = his_unit[his_seg : his_seg + his_win_size][selected_ftr_name]
                dist_list += [fastdtw(on_array, his_array)[0]]
            min_his_seg = np.argsort(dist_list)[0]
            dtw_dist_df = dtw_dist_df.append(pd.DataFrame([[on_id, his_id, on_seg, min_his_seg+cur_his_seg, on_len - on_seg - on_win_size, his_len - (min_his_seg+cur_his_seg) - his_win_size,dist_list[min_his_seg]]],columns = dist_info_col))
            #dtw_dist_dict[on_id][his_id][on_seg] = (min_his_seg+cur_his_seg, dist_list[min_his_seg])
            cur_his_seg = max(min_his_seg+cur_his_seg - 20, 0)
            #print(his_id, on_seg, cur_his_seg)
            
    t1= time.time()
    print('unit_id:',on_id,'segs:',n_segs,'time:',t1-t0,'avg:',(t1-t0)/n_segs)
t_end = time.time()
print('tot_time:',t_end-t_begin)

--- 11 ---
unit_id: 11 segs: 22 time: 1174.872921705246 avg: 53.40331462296572
--- 20 ---
unit_id: 20 segs: 21 time: 687.7282238006592 avg: 32.748963038126625
--- 17 ---
unit_id: 17 segs: 25 time: 1381.0634405612946 avg: 55.242537622451785
--- 41 ---
unit_id: 41 segs: 19 time: 1004.3649101257324 avg: 52.861311059249076
--- 42 ---
unit_id: 42 segs: 17 time: 815.7186269760132 avg: 47.98344864564783
--- 97 ---
unit_id: 97 segs: 18 time: 981.8578979969025 avg: 54.54766099982791
--- 80 ---
unit_id: 80 segs: 16 time: 554.0121629238129 avg: 34.625760182738304
--- 45 ---
unit_id: 45 segs: 13 time: 752.2639636993408 avg: 57.86645874610314
--- 48 ---
unit_id: 48 segs: 21 time: 1229.6224746704102 avg: 58.553451174781436
--- 39 ---
unit_id: 39 segs: 10 time: 361.2597219944 avg: 36.12597219944
--- 36 ---
unit_id: 36 segs: 13 time: 493.9931399822235 avg: 37.99947230632488
--- 14 ---
unit_id: 14 segs: 16 time: 583.7238590717316 avg: 36.48274119198322
--- 31 ---
unit_id: 31 segs: 21 time: 1272.7818410

In [53]:
dtw_dist_df.head()

,on_id,his_id,on_seg,his_seg,on_RUL,his_RUL,dist
0,81,3,0,94,210,55,10.211874
0,81,3,10,95,200,54,7.321320
0,81,3,20,95,190,54,7.304314
0,81,3,30,100,180,49,5.980598
0,81,3,40,100,170,49,6.067938


In [26]:
dtw_dist_df.to_csv(res_file+'/dtw_dist_'+str(random_state)+'.csv',index=False,header=True)

In [73]:
tmp_df = dtw_dist_df[(dtw_dist_df['on_id']==81) &(dtw_dist_df['on_seg']==70)].sort_values(by='dist')
tmp_df

,on_id,his_id,on_seg,his_seg,on_RUL,his_RUL,dist
364,81,60,70,30,140,112,0.228318
1435,81,72,70,92,140,91,0.369057
784,81,88,70,101,140,82,0.548866
889,81,71,70,0,140,178,0.552363
1225,81,86,70,87,140,161,0.694432
1288,81,29,70,37,140,96,0.708942
259,81,55,70,49,140,114,0.711918
280,81,96,70,170,140,136,0.727664
868,81,67,70,103,140,180,0.736956
406,81,35,70,67,140,84,0.810296


In [79]:
sum(tmp_df['his_RUL']*np.exp(-tmp_df['dist']))/sum(np.exp(-tmp_df['dist']))

128.6649844554795

In [143]:
(tmp_df['his_RUL'] - tmp_tg)*np.exp(-tmp_df['dist'])*np.exp(-1/tmp_df['time_gap']+0.0001)

26390        11.1166
26391        14.9493
26747        2.42006
26748        5.58862
27357       -29.3959
26538        7.10324
26769        7.09178
27440       -9.42744
26810       -27.4729
27331        36.1392
26811       -23.3253
26770        10.8283
27356       -30.0817
26749        9.22018
26307       -12.0109
26539         9.7484
26746         8.2071
26560       -12.3054
26148        23.9298
26141        37.2302
26392        12.6304
26685        6.27726
26833       -33.8329
27345        20.9364
25971        23.9345
26905        12.6359
27262        31.0048
26745         4.0437
26149        22.1376
26485        14.3204
            ...     
26975   -2.58717e-39
27458   -9.00301e-41
26976    -1.4952e-39
27459    1.45687e-40
26601    6.36959e-40
26935   -6.34604e-41
26875   -2.09698e-40
26328    1.50133e-40
26344    1.27732e-40
26933   -9.94775e-41
26974   -2.07415e-40
26934   -4.24549e-41
27457   -2.68089e-41
26600    8.59706e-42
26932   -9.46504e-42
26874   -3.51857e-42
26327    1.90

In [50]:
cut_percents = [0.5,0.7,0.9]
dtw_score = []
for k in param_grid['K']:
    for alpha in param_grid['alpha']:
        res_info_col = ['on_id','on_seg','y_pred','y_true']
        res_df = pd.DataFrame(columns=res_info_col)
        for on_id in test_id:
            n_seg = len(test_df.loc[on_id])
            for cut_per in cut_percents:
                cur_seg = int(n_seg * cut_per/10)*10 - on_win_size
                tmp_df = dtw_dist_df[(dtw_dist_df['on_id']==on_id) &(dtw_dist_df['on_seg']==cur_seg)].sort_values(by='dist')[:k]
                tmp_rul = sum(tmp_df['his_RUL']*np.exp(-alpha * tmp_df['dist'])/sum(np.exp(-alpha * tmp_df['dist'])))
                #print(cur_seg,tmp_rul, cur_seg+tmp_rul, n_seg - cur_seg - on_win_size)
                res_df= res_df.append(pd.DataFrame([[on_id, cur_seg, tmp_rul, n_seg -cur_seg - on_win_size]],columns = res_info_col))
        dtw_score += [(k, alpha, RHS_score( res_df['y_true'], res_df['y_pred'])/len(res_df['y_pred']))]

In [51]:
dtw_score

[(50, 0, 15.981904912787066),
 (50, 0.5, 10.350215450953211),
 (50, 1, 98.07284389248544),
 (50, 1.5, 349.23805774347414),
 (50, 2, 810.6487973168447),
 (60, 0, 25.729555797930363),
 (60, 0.5, 10.359121261673597),
 (60, 1, 98.07778368830434),
 (60, 1.5, 349.24044146122577),
 (60, 2, 810.6494916761168),
 (70, 0, 42.230516837153154),
 (70, 0.5, 10.350853773820507),
 (70, 1, 98.07704865652961),
 (70, 1.5, 349.2403868833245),
 (70, 2, 810.6494859020087),
 (80, 0, 66.07779456623682),
 (80, 0.5, 10.354131148342173),
 (80, 1, 98.07732170860452),
 (80, 1.5, 349.2404093173371),
 (80, 2, 810.6494875337974),
 (90, 0, 66.07779456623682),
 (90, 0.5, 10.354131148342173),
 (90, 1, 98.07732170860452),
 (90, 1.5, 349.2404093173371),
 (90, 2, 810.6494875337974),
 (100, 0, 66.07779456623682),
 (100, 0.5, 10.354131148342173),
 (100, 1, 98.07732170860452),
 (100, 1.5, 349.2404093173371),
 (100, 2, 810.6494875337974)]

In [165]:
def rul_threshold(x):
    MAX_RUL = 130
    if x >= MAX_RUL:
        return MAX_RUL
    else:
        return x

In [167]:
MAX_RUL = 130
#res_df['y_pred']
res_df['y_pred'] = res_df['y_pred'].apply(lambda x : rul_threshold(x))
res_df['y_true'] = res_df['y_true'].apply(lambda x : rul_threshold(x))
#np.sqrt(sum((res_df['y_pred'] - res_df['y_true'])**2))

In [170]:
np.sqrt(sum((res_df['y_pred'] - res_df['y_true'])**2))

130.30938104203923

In [146]:
cut_percents = [0.5,0.7,0.9]
for on_id in test_id:
    n_seg = len(test_df.loc[on_id])
    print(n_seg)
    for cut_per in cut_percents:
        cur_seg = int(n_seg * cut_per) - 30
        tmp_df = dtw_dist_df[(dtw_dist_df['on_id']==on_id) &(dtw_dist_df['on_seg']<=cur_seg)].sort_values(by='dist')
        tmp_df['time_gap'] = (cur_seg - tmp_df['on_seg']).astype(np.float64)
        tmp_rul = sum((tmp_df['his_RUL'] - tmp_tg)*np.exp(-tmp_df['dist'])*np.exp(-1/tmp_df['time_gap']+0.0001)/sum(np.exp(-tmp_df['dist'])+1)/max(1,sum(np.exp(-1/tmp_df['time_gap']))))
        print(cur_seg,tmp_rul, cur_seg+tmp_rul, n_seg - cur_seg + 30)

240
90 nan nan 180
138 nan nan 132
186 nan nan 84
188
64 nan nan 154
101 nan nan 117
139 nan nan 79
195
67 nan nan 158
106 nan nan 119
145 nan nan 80
214
77 nan nan 167
119 nan nan 125
162 nan nan 82
258
99 nan nan 189
150 nan nan 138
202 nan nan 86
195
67 nan nan 158
106 nan nan 119
145 nan nan 80
170
55 nan nan 145
88 nan nan 112
123 nan nan 77
293
116 nan nan 207
175 nan nan 148
233 nan nan 90
137
38 nan nan 129
65 nan nan 102
93 nan nan 74
202
71 nan nan 161
111 nan nan 121
151 nan nan 81
155
47 nan nan 138
78 nan nan 107
109 nan nan 76
188
64 nan nan 154
101 nan nan 117
139 nan nan 79
137
38 nan nan 129
65 nan nan 102
93 nan nan 74
195
67 nan nan 158
106 nan nan 119
145 nan nan 80
213
76 nan nan 167
119 nan nan 124
161 nan nan 82
200
70 nan nan 160
110 nan nan 120
150 nan nan 80
191
65 nan nan 156
103 nan nan 118
141 nan nan 80
158
49 nan nan 139
80 nan nan 108
112 nan nan 76
199
69 nan nan 160
109 nan nan 120
149 nan nan 80
240
90 nan nan 180
138 nan nan 132
186 0.001268277980446

In [128]:
tmp_df['time_gap']

364     2.1
799     8.1
344     1.1
800     7.1
1349    4.1
1290    0.1
282     0.1
218     1.1
1200    6.1
1221    6.1
1348    5.1
363     3.1
365     1.1
1435    2.1
1223    4.1
258     3.1
1222    5.1
114     0.1
1226    1.1
785     1.1
572     4.1
1080    0.1
1227    0.1
1201    5.1
593     4.1
1352    1.1
680     1.1
1434    3.1
568     8.1
1433    4.1
       ... 
506     7.1
1617    9.1
1136    7.1
1619    7.1
1095    6.1
761     4.1
505     8.1
1135    8.1
1618    8.1
488     4.1
1092    9.1
1035    3.1
1094    7.1
1093    8.1
760     5.1
487     5.1
1034    4.1
759     6.1
486     6.1
756     9.1
758     7.1
1033    5.1
483     9.1
757     8.1
485     7.1
484     8.1
1032    6.1
1029    9.1
1031    7.1
1030    8.1
Name: time_gap, Length: 800, dtype: object

In [114]:
pd.DataFrame(cur_seg - tmp_df['on_seg'])

,on_seg
26390,146
26391,136
26747,146
26748,136
27357,136
26538,136
26769,136
27440,146
26810,146
27331,186


In [133]:
np.exp(-1/2)

0.6065306597126334